In [ ]:
# Display metadata
import os
import glob
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

pd.set_option('display.max_colwidth', None)
non_partitioned_table_path = '/home/hive/warehouse/transformed.db/iceberg_demo'
partitioned_table_path = '/home/hive/warehouse/transformed.db/iceberg_partitioned_demo'


catalog_name = "iceberg_catalog"
database_name = "transformed"
table_name = "iceberg_demo"
table_name_partitioned = "iceberg_partitioned_demo"

def display_metadata(demo_tbl_df, table_path):

    
    metadata_files_path = os.path.join(table_path, 'metadata/*.json')
    meta_data_files = sorted(glob.iglob(metadata_files_path), key=os.path.getctime, reverse=True)

    data_files_path = os.path.join(table_path, 'data/*')
    data_files = sorted(glob.iglob(data_files_path), key=os.path.getctime, reverse=True)

    print("---------------- Data Files List -----------------------")
    data_files_df = spark.createDataFrame(data_files, StringType(), )
    display(data_files_df.toPandas())

    print("---------------- Meta Data Files List -----------------------")
    metadata_files_df = spark.createDataFrame(meta_data_files, StringType())
    display(metadata_files_df.toPandas())

    print("---------------- Meta Data Json -----------------------")
    latest_metadata_file = meta_data_files[0]
    metadata_df = spark.read.format("json").option("multiLine", True).load(latest_metadata_file).select(
        F.col("current-schema-id"),
        F.col("current-snapshot-id"),
        F.col("current-schema-id"),
        F.col("partition-spec"),
        F.col("schemas"),
        F.explode(F.col("snapshots")).alias("snapshot")
    ).select(
        F.col("current-schema-id"),
        F.col("current-snapshot-id"),
        F.col("current-schema-id"),
        F.col("partition-spec"),
        F.col("snapshot.snapshot-id"),
        F.col("snapshot.manifest-list"),
        F.explode(F.col("schemas")).alias("schema")
    ).select(
        F.col("current-snapshot-id"),
        F.col("snapshot-id"),
        F.col("partition-spec"),
        F.col("current-schema-id"),
        F.col("schema.schema-id"),
        F.col("schema.fields").alias("schema"),
        F.col("manifest-list")
    ).filter(F.col("current-schema-id") == F.col("schema-id")).filter(F.col("current-snapshot-id") == F.col("snapshot-id"))
    #metadata_df.printSchema()
    display(metadata_df.toPandas())

    print("---------------- Manifest List Avro -----------------------")
    manifest_list_files = list(metadata_df.select(F.col("manifest-list")).toPandas()['manifest-list'])
    print(manifest_list_files)
    manifest_list_df = spark.read.format("avro").load(manifest_list_files).select(
        F.col("added_snapshot_id"),
        F.col("added_data_files_count"),
        F.col("existing_data_files_count"),
        F.col("deleted_data_files_count"),
        F.col("added_rows_count"),
        F.col("deleted_rows_count"),
        F.col("manifest_path")
    )
    display(manifest_list_df.toPandas())

    print("---------------- Manifest Avro -----------------------")
    manifest_files = list(manifest_list_df.select(F.col("manifest_path")).toPandas()['manifest_path'])
    print(manifest_list_files)
    manifest_df = spark.read.format("avro").load(manifest_files).select(
        F.col("*")
    )
    display(manifest_df.toPandas())




In [ ]:
display(spark.sql("show catalogs").toPandas())
spark.sql("CREATE DATABASE transformed")
display(spark.sql("show databases in iceberg_catalog").toPandas())

In [20]:
# Create Demo Iceberg Table
fact_sales_df = spark.read.format("parquet").load("/home/datalake/transformed.fact_sales.parquet/")
fact_sales_df.count()
spark.sql(f"DROP TABLE IF EXISTS {catalog_name}.{database_name}.{table_name}")
os.system(f"rm -rf {non_partitioned_table_path}")

fact_sales_df.writeTo(f"{catalog_name}.{database_name}.{table_name}").create()
print("Table created.")

Table created.


In [21]:
# Read Iceberg Table and Display Metadata
demo_iceberg_df = spark.table(f"{catalog_name}.{database_name}.{table_name}")
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

---------------- Data Files List -----------------------


,value
0,/home/hive/warehouse/transformed.db/iceberg_demo/data/00002-780-fa2b74af-f62a-4407-a0ce-1a650046fd04-00001.parquet
1,/home/hive/warehouse/transformed.db/iceberg_demo/data/00001-779-eb3b6e8b-516b-42d6-997a-e6c5b4c78c38-00001.parquet
2,/home/hive/warehouse/transformed.db/iceberg_demo/data/00000-778-dd77f2b0-d6d9-4db5-8629-f938ee4f64f4-00001.parquet


---------------- Meta Data Files List -----------------------


,value
0,/home/hive/warehouse/transformed.db/iceberg_demo/metadata/00000-966bdb9c-58de-4be7-b966-a371f28713cf.metadata.json


---------------- Meta Data Json -----------------------


,current-snapshot-id,snapshot-id,partition-spec,current-schema-id,schema-id,schema,manifest-list
0,3345030398619684425,3345030398619684425,[],0,0,"[(1, order_date, False, timestamptz), (2, shipping_country, False, string), (3, order_item_id, False, int), (4, quantity, False, int), (5, unit_price, False, decimal(18, 2)), (6, subtotal, False, decimal(29, 2)), (7, ctrl_load_date, False, timestamptz), (8, ctrl_row_hash, False, string), (9, product_hash_key, False, string), (10, order_hash_key, False, string), (11, seller_hash_key, False, string), (12, supplier_hash_key, False, string)]",file:/home/hive/warehouse/transformed.db/iceberg_demo/metadata/snap-3345030398619684425-1-543eeb05-c78d-4ab4-a392-2a49520419d8.avro


---------------- Manifest List Avro -----------------------
['file:/home/hive/warehouse/transformed.db/iceberg_demo/metadata/snap-3345030398619684425-1-543eeb05-c78d-4ab4-a392-2a49520419d8.avro']


,added_snapshot_id,added_data_files_count,existing_data_files_count,deleted_data_files_count,added_rows_count,deleted_rows_count,manifest_path
0,3345030398619684425,3,0,0,955,0,file:/home/hive/warehouse/transformed.db/iceberg_demo/metadata/543eeb05-c78d-4ab4-a392-2a49520419d8-m0.avro


---------------- Manifest Avro -----------------------
['file:/home/hive/warehouse/transformed.db/iceberg_demo/metadata/snap-3345030398619684425-1-543eeb05-c78d-4ab4-a392-2a49520419d8.avro']


,status,snapshot_id,data_file
0,1,3345030398619684425,"(file:/home/hive/warehouse/transformed.db/iceberg_demo/data/00000-778-dd77f2b0-d6d9-4db5-8629-f938ee4f64f4-00001.parquet, PARQUET, (), 318, 37355, 67108864, [(1, 1943), (2, 781), (3, 281), (4, 336), (5, 953), (6, 1151), (7, 99), (8, 6408), (9, 5494), (10, 5954), (11, 5421), (12, 4952)], [(1, 318), (2, 318), (3, 318), (4, 318), (5, 318), (6, 318), (7, 318), (8, 318), (9, 318), (10, 318), (11, 318), (12, 318)], [(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0)], [], [(1, bytearray(b'\x00\xa7I\xd4P\xa8\x05\x00')), (2, bytearray(b'Afghanistan')), (3, bytearray(b'\x01\x00\x00\x00')), (4, bytearray(b'\x01\x00\x00\x00')), (5, bytearray(b'y')), (6, bytearray(b'\x00\xf5')), (7, bytearray(b'fa\x82\xc7\xad\xe1\x05\x00')), (8, bytearray(b'01b22eef8f26ec48')), (9, bytearray(b'00c3dd5bacb189ae')), (10, bytearray(b'00572e1c606c9646')), (11, bytearray(b'01a3319b69a50fb5')), (12, bytearray(b'00c3dd5bacb189ae'))], [(1, bytearray(b'@\x89\xeb\xd4\\\xe1\x05\x00')), (2, bytearray(b'Yemen')), (3, bytearray(b'\n\x00\x00\x00')), (4, bytearray(b'\x14\x00\x00\x00')), (5, bytearray(b'&\xf1')), (6, bytearray(b'\x02\xd98')), (7, bytearray(b'fa\x82\xc7\xad\xe1\x05\x00')), (8, bytearray(b'fe8e38c3208c098d')), (9, bytearray(b'fff740e390c35ca2')), (10, bytearray(b'fff740e390c35ca2')), (11, bytearray(b'fff740e390c35ca2')), (12, bytearray(b'fff740e390c35ca2'))], None, [4], 0)"
1,1,3345030398619684425,"(file:/home/hive/warehouse/transformed.db/iceberg_demo/data/00001-779-eb3b6e8b-516b-42d6-997a-e6c5b4c78c38-00001.parquet, PARQUET, (), 319, 36942, 67108864, [(1, 1976), (2, 867), (3, 281), (4, 336), (5, 947), (6, 1151), (7, 113), (8, 6437), (9, 5170), (10, 6086), (11, 5241), (12, 4753)], [(1, 319), (2, 319), (3, 319), (4, 319), (5, 319), (6, 319), (7, 319), (8, 319), (9, 319), (10, 319), (11, 319), (12, 319)], [(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0)], [], [(1, bytearray(b'\x80A\xe0UU\xa8\x05\x00')), (2, bytearray(b'Afghanistan')), (3, bytearray(b'\x01\x00\x00\x00')), (4, bytearray(b'\x01\x00\x00\x00')), (5, bytearray(b'f')), (6, bytearray(b'\x01J')), (7, bytearray(b'fa\x82\xc7\xad\xe1\x05\x00')), (8, bytearray(b'00f1e52c2a9bbe52')), (9, bytearray(b'0016865a5a7fc171')), (10, bytearray(b'00c3dd5bacb189ae')), (11, bytearray(b'0016865a5a7fc171')), (12, bytearray(b'01e0f5fbe130458f'))], [(1, bytearray(b'\xc0\xae\xce\xc0\xc9\xe1\x05\x00')), (2, bytearray(b'Zambia')), (3, bytearray(b'\n\x00\x00\x00')), (4, bytearray(b'\x14\x00\x00\x00')), (5, bytearray(b'N ')), (6, bytearray(b'\x02\xbfr')), (7, bytearray(b'\xd1\x82E\xeb\xc3\xe1\x05\x00')), (8, bytearray(b'ff4daebb517695dc')), (9, bytearray(b'fff740e390c35ca2')), (10, bytearray(b'fff740e390c35ca2')), (11, bytearray(b'fff740e390c35ca2')), (12, bytearray(b'ff94ee1b1fc531bd'))], None, [4], 0)"
2,1,3345030398619684425,"(file:/home/hive/warehouse/transformed.db/iceberg_demo/data/00002-780-fa2b74af-f62a-4407-a0ce-1a650046fd04-00001.parquet, PARQUET, (), 318, 36923, 67108864, [(1, 1964), (2, 904), (3, 281), (4, 336), (5, 942), (6, 1146), (7, 99), (8, 6416), (9, 5291), (10, 5966), (11, 5232), (12, 4766)], [(1, 318), (2, 318), (3, 318), (4, 318), (5, 318), (6, 318), (7, 318), (8, 318), (9, 318), (10, 318), (11, 318), (12, 318)], [(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0)], [], [(1, bytearray(b'\xc0\xd2]{6\xa8\x05\x00')), (2, bytearray(b'Albania')), (3, bytearray(b'\x01\x00\x00\x00')), (4, bytearray(b'\x01\x00\x00\x00')), (5, bytearray(b'o')), (6, bytearray(b'\x00\x89')), (7, bytearray(b'fa\x82\xc7\xad\xe1\x05\x00')), (8, bytearray(b'0184f6ac06497a56')), (9, bytearray(b'0016865a5a7fc171')), (10, bytearray(b'0016865a5a7fc171')), (11, bytearray(b'0016865a5a7fc171')), (12, bytearray(b'00f1797222d01d85'))], [(1, bytearray(b'\x80\r\xae\xfdW\xe1\x05\x00')), (2, bytearray(b'Zimbabwe')), (3, bytearray(b'\n\x00\x00\x00')), (4,

In [ ]:
# Update a record in Iceberg Table
display(spark.sql(f"SELECT * FROM {catalog_name}.{database_name}.{table_name} WHERE order_hash_key = '733398b50242a8734489a906a12a793f' and order_item_id = '1'").withColumn("file_name", F.input_file_name()).toPandas())
iceberg_update_output = spark.sql(f"""
UPDATE {catalog_name}.{database_name}.{table_name}
SET shipping_country = 'USA'
WHERE order_hash_key = '733398b50242a8734489a906a12a793f' and order_item_id = '1'
""")
spark.catalog.clearCache()
demo_iceberg_df
display(spark.sql(f"SELECT * FROM {catalog_name}.{database_name}.{table_name} WHERE order_hash_key = '733398b50242a8734489a906a12a793f' and order_item_id = '1'").withColumn("file_name", F.input_file_name()).toPandas())

In [ ]:
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

In [ ]:
# Schema Evolution - DELETE a recond
spark.sql(f"""DELETE FROM {catalog_name}.{database_name}.{table_name} WHERE order_hash_key = '733398b50242a8734489a906a12a793f' and order_item_id = '1'""")
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

In [ ]:
# Schema Evolution - ADD a Column
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
spark.sql(f"""ALTER TABLE {catalog_name}.{database_name}.{table_name} ADD  COLUMNS (
                shipping_city string
                )""")
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

In [ ]:
# Schema Evolution - Remove a Column
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
spark.sql(f"""ALTER TABLE {catalog_name}.{database_name}.{table_name} DROP COLUMN ctrl_load_date""")
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

In [ ]:
# Schema Evolution - Rename a Column
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
spark.sql(f"""ALTER TABLE {catalog_name}.{database_name}.{table_name} RENAME COLUMN subtotal TO sub_total""")
display(spark.sql(f"DESCRIBE {catalog_name}.{database_name}.{table_name}").toPandas())
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=non_partitioned_table_path)

In [ ]:
spark.catalog.clearCache()
demo_iceberg_final = spark.table(f"{catalog_name}.{database_name}.{table_name}")
display(demo_iceberg_final.limit(5).toPandas())

In [ ]:
# Create Partitioned Iceberg Table

fact_sales_df = spark.read.format("parquet").load("/home/datalake/transformed.fact_sales.parquet/")
fact_sales_df.count()
spark.sql(f"DROP TABLE IF EXISTS {catalog_name}.{database_name}.{table_name_partitioned}")
os.system(f"rm -rf {partitioned_table_path}")

fact_sales_df.sortWithinPartitions("ctrl_load_date").writeTo(f"{catalog_name}.{database_name}.{table_name_partitioned}").partitionedBy(F.days("ctrl_load_date")).createOrReplace()
print("Table created.")
display_metadata(demo_tbl_df=demo_iceberg_df, table_path=partitioned_table_path)

In [ ]:
display(spark.sql(f"SELECT * FROM {catalog_name}.{database_name}.{table_name_partitioned} LIMIT 10").show())